# **Assignment I: Integrators**

### 1. **Introduction**

Molecular Dynamics (MD) simulates the time evolution of (bio)physical systems by integrating the (classical) equations of motion:

$$\frac{d\mathbf{q}_{i}}{dt} = \frac{\partial\mathscr{H}}{\partial\mathbf{p}_{i}}  ,$$
$$\frac{d\mathbf{p}_{i}}{dt} = -\frac{\partial\mathscr{H}}{\partial\mathbf{q}_{i}} ,$$

where $\mathscr{H}$ is Hamiltonian of the $N$-particles system, $\mathbf{p} = \{\mathbf{p}_{i}\in\mathbb{R}^3\}_{i=1, \dots, N}$ the particle momenta, and $\mathbf{q} = \{\mathbf{q}_{i}\in\mathbb{R}^3\}_{i=1, \dots, N}$ the particle coordinates. The Hamiltonian $\mathscr{H}$ is defined as the sum of kinetic energy and the potential energy function $U: \mathbf{q} \in \mathbb{R}^{3N} \mapsto \mathbb{R}$:

$$\mathscr{H}(\mathbf{p}, \mathbf{q}) \coloneqq \frac{1}{2}\sum_{i=1}^{N} \frac{\mathbf{p}_{i} \cdot \mathbf{p}_{i}^\top}{2{m}_{i}} + U(\mathbf{q}) ,$$

In this assignment, you will implement the numerical algorithm for integrating the equations of motion that facilitate the dynamical evolution of noble gas particles interacting through a Lennard-Jones (LJ) potential function.

### 2. **Objectives**

- **Derive a theoretical algorithm for dynamical integrator with thermostat (temperature control);**

- **Implement and test the integrator on a box of noble gas particles based on the interfaces provided.**

### 3. **Instructions**

The integrator implementation is based on our in-house MD package `MDPy` for simulating LJ particles with periodic boundary conditions (PBC)

The `MDPy` uses the `NumPy` infrastructure for matrix operations.

It contains the following modules:

| Modules       | Description                                                                  |
| :---          | :---                                                                         |
| `box`         | implements the PBC box.                                                      |
| `potentials`  | implements the interfaces for potential functions and the 12-6 LJ potential. |
| `integrators` | implements the interfaces for integrators and the ISP Langevin integrator.   |
| `system`      | implements the MD system that holds the box and the particle information.    |

In this assignment, you will implement the necessary functions for an integrator by sub-classing the `mdpy.integrator.Integrator()` abstract class.

---

### 3. **Solved Example**

A solved example is provided for your reference, which derives implements the ISP Langevin integrator.

##### 3.1 **Derive the integration algorithm**

The underdamped Langevin equation provides a concise approach for maintaining the temperature through the Dissipation-Fluctuation Theorem:

$$m_i \frac{dv_i(t)}{dt} = -m_i \gamma v_i(t) + F_i + \xi_i(t), $$

where $m_i$, $v_i$, and $x_i$ are the mass, the velocity, and the coordinate of the $i$-th particle, respectively; $F_i$ is the interacting force (i.e., the negative potential gradient) and $\xi_i\sim\left( 2m_i\gamma k_BT \right)^{\frac{1}{2}}\mathcal{N}(0, 1)$ the thermo noise, both acting on the $i$-th particle; and $\gamma$ denotes the homogeneous friction coefficient.

Rearrange the Langevin equation and integrate by part:

$$\frac{dv_i(t)}{dt} + \gamma v_i(t) = \frac{1}{m_i}F_i + \frac{1}{m_i}\xi_i(t) ,$$

$$\exp(\gamma t)\frac{dv_i(t)}{dt} + \exp(\gamma t)\gamma v_i(t) = \exp(\gamma t)\frac{1}{m_i}F_i + \exp(\gamma t)\frac{1}{m_i}\xi_i(t) ,$$

$$d\exp(\gamma t)v_i(t) = \exp(\gamma t)\frac{1}{m_i}F_i + \exp(\gamma t)\frac{1}{m_i}\xi_i(t) ,$$

$$\int_{\tau=t}^{\tau=t+dt}d\exp(\gamma\tau)v_i(\tau) = \underbrace{\frac{1}{m_i} \left(\int_t^{t+dt}d\tau\exp(\gamma\tau)F_i\right)}_{\text{drift}} + \underbrace{\frac{1}{m_i} \left(\int_t^{t+dt}d\tau\exp(\gamma\tau)\xi_i(\tau)\right)}_{\text{diffusion}} ,$$

We derive each term in the above equation.

The L.H.S.: 

$$\int_{\tau=t}^{\tau=t+dt}d\exp(\gamma\tau)v_i(\tau) = \exp(\gamma(t+dt))v_i(t+dt) - \exp(\gamma t)v_i(t), $$

The R.H.S., the drift term assumes the $F_i$ does not change for $dt$ sufficiently small:

$$\frac{1}{m_i} \left(\int_t^{t+dt}d\tau\exp(\gamma\tau)F_i\right) = \frac{1}{\gamma m_i} \int_t^{t+dt}d\left(\exp(\gamma\tau)F_i\right) = \frac{1}{\gamma m_i} \left(\exp(\gamma(t+dt)) - \exp(\gamma t)\right) F_i, $$

The R.H.S.,  the diffusion term is a scaled Gaussian noise parameterized by its mean:

$$\left\langle\frac{1}{m_i}\int_t^{t+dt}d\tau\exp(\gamma\tau)\xi_i(\tau)\right\rangle = \frac{1}{m_i}\int_t^{t+dt}d\tau\exp(\gamma\tau)\overbrace{\langle\xi_i(\tau)\rangle}^0 = 0, $$

and variance:

$$\left\langle\frac{1}{m_i}\int_t^{t+dt}d\tau\exp(\gamma\tau)\xi_i(\tau)\frac{1}{m_i}\int_t^{t+dt}d\tau'\exp(\gamma\tau')\xi_i(\tau')\right\rangle = \frac{1}{m_i^2}\int_t^{t+dt}d\tau\int_t^{t+dt}d\tau'\exp(\gamma(\tau+\tau'))\langle\xi_i(\tau)\xi_i(\tau')\rangle $$

$$ = \frac{1}{m_i^2}\int_t^{t+dt}d\tau\int_t^{t+dt}d\tau'\exp(\gamma(\tau+\tau'))2m_i\gamma k_BT\delta(\tau-\tau') $$

$$ = \frac{k_BT}{m_i}\int_t^{t+dt}d(2\gamma\tau)\exp(2\gamma\tau) $$

$$ = \frac{k_BT}{m_i}(\exp(2\gamma(t+dt))-\exp(2\gamma t)), $$

By substituting each components, the velocity propagation step is:

$$\exp(\gamma(t+dt))v_i(t+dt) = \exp(\gamma t)v_i(t) + \frac{1}{m_i\gamma} \exp(\gamma(t+dt))-\exp(\gamma t) F_i + \left( \frac{k_BT}{m_i}(\exp(2\gamma(t+dt)) - \exp(2\gamma t)) \right)^{\frac{1}{2}} \mathcal{R}(t) ,$$

$$v_i(t+dt) = \exp(-\gamma dt)v_i(t) + \frac{1-\exp(-\gamma dt)}{m_i\gamma}F_i + \left( \frac{k_BT}{m_i}(1-\exp(-2\gamma dt)) \right)^{\frac{1}{2}} mathcal{R}(t) ,$$

$$v_i(t+dt) = A v_i(t) + \frac{1}{m_i}B F_i + C\left( \frac{k_BT}{m_i} \right)^{\frac{1}{2}} \mathcal{R}(t), $$

where we have denote for convenience:

$$A \coloneqq \exp(-\gamma dt) ,$$

$$B \coloneqq \frac{1}{\gamma}(1-\exp(-\gamma dt)) ,$$

$$C \coloneqq \left( (1-\exp(-2\gamma dt)) \right)^{\frac{1}{2}} ,$$

We use the `leap-frog` scheme, the velocities are propagated at half-times steps ($\frac{1}{2}dt$), the integration reads:

$$\text{Step I}:   \ \ v_i\left(t+\frac{1}{2}dt\right) = A v_i\left(t-\frac{1}{2}dt\right) + \frac{1}{m_i}B F_i + C\left( \frac{k_BT}{m_i} \right)^{\frac{1}{2}} \mathcal{R}\left(t-\frac{1}{2}dt\right) ,$$

$$\text{Step II}:  \ \ x_i\left(t+dt\right) = x_i(t) + v_i\left(t+\frac{1}{2}dt\right) dt ,$$

$$\text{Step III}: \ \ v_i\left(t+\frac{1}{2}dt\right) = \frac{1}{dt} (x_i(t+dt) - x_i(t)),$$

We are now ready to implement the integrator.

**References**


##### 3.2. **Implement the integrator**


In [4]:
import numpy as np

import mdpy.utils
import mdpy.system
import mdpy.integrators

class LangevinISPIntegrator(mdpy.integrators.Integrator):

  def __init__(self, 
               system: mdpy.system.System, 
               dt:    float, 
               T:     float, 
               gamma: float, ):
    r"""The constructor for the Langevin ISP integrator.
    
      Args:
        system (mdpy.system.System): The `MDPy` system bounded to this integrator.
        dt     (float): The timestep size (in ps).
        T      (float): The temperature (in K).
        gamma  (float): The friction coefficient (in 1/ps).
    """
    super().__init__(system)
    # Control paramters.
    self.dt = dt
    self.T = T
    self.gamma = gamma
    self.kT = mdpy.utils.IDEAL_GAS_CONSTANT * self.T  # This is the k_BT
    # Compute the A, B, and C.
    self.A  = np.exp(-self.gamma * self.dt)
    self.B  = np.exp(1. - np.exp(-self.gamma * self.dt)) / self.gamma
    self.C  = np.sqrt(1. - np.exp(-2. * self.gamma * self.dt))
  
  def step(self) -> None:
    r"""Advance the integration by ONE time step."""
    # extract states from the system.
    m = self.system.topology.masses  # [N, 1], the mass        m
    x = self.system.coordinates      # [N, 3], the coordinates x(t)
    v = self.system.velocities       # [N, 3], the velocities  v(t-0.5dt)
    f = self.system.compute_forces() # [N, 3], the potential forces F
    r = np.random.randn(*f.shape)    # [N, 3], the random forces    R
    
    # integration.
    ## Step I:
    v_half = self.A*v + self.B*f/m + self.C*r*np.sqrt(self.kT/m)  
    ## Step II:
    x_full_next = x + v_half*self.dt
    ## Step III:
    v_half_next = (x_full_next - x) / self.dt
    
    # update states to the system.
    self.system.velocities  = v_half_next # Update velocities  v(t+0.5dt)
    self.system.coordinates = x_full_next # Update coordinates x(t+dt).

  def initialize_velocities(self, temperature: float) -> None:
    r"""Initialize the particle velocities per Maxwellian at the designated temperature.
    
      Args:
        temperature (float): The temperature (in K).
    """
    kT = mdpy.utils.IDEAL_GAS_CONSTANT * temperature
    # extract system states.
    m = self.system.topology.masses  # [N, 1], the mass
    f = self.system.compute_forces() # [N, 3], the potential forces F

    # initialize v(t).
    v = np.random.normal(loc=0., scale=kT/m, size=(self.system.topology.num_particles, 3))
    
    # shift v(t) to v(t-.5dt).
    v = v - 0.5*self.dt*f/m
    
    # update system states.
    self.system.velocities = v  # Update velocities v(t-0.5dt)

  def compute_kinetic_energy(self) -> float:
    r"""Compute the kinetic energy (in kcal/mol) of the system using the shifted velocities v(t).
    
      Returns:
        kinetic_energy (float): The kinetic energy (in kcal/mol) at v(t).
    """
    # extract system states.
    m = self.system.topology.masses
    v = self.system.velocities       # [N, 3], the velocities v(t-0.5dt)
    f = self.system.compute_forces() # [N, 3], the potential forces F

    # shift v(t-.5dt) to v(t).
    v = v + 0.5*self.dt*f/m
    
    # K = sum(mv^2/2)
    kinetic_energy = np.sum(0.5*m*v**2)
    return kinetic_energy

##### 3.3. **Test the integrator**

Note that the `mdpy.box.PBCBox().wrap()` method will compute the principal cell coordinates.

In [5]:
import mdpy.box
import mdpy.topology
import mdpy.potentials
import mdpy.system

# Elementary objects of an MD system.
box         = mdpy.box.PBCBox(xdim=50., ydim=50., zdim=50.)
topology    = mdpy.topology.HomogeneousIdealGas(num_particles=200, mass=1.)
coordinates = box.wrap(coordinates=np.random.randn(200, 3)*100.)
potential   = mdpy.potentials.LJ126(sigma=1., epsilon=1.)

# The MD system.
system = mdpy.system.System(box=box, topology=topology, coordinates=coordinates)
system.add_potential(potential)

# Visualize the simulation box using `MDTraj` and `NGLView`.
import mdtraj
import nglview

traj = mdtraj.Trajectory(xyz=np.asarray([system.coordinates]), topology=system.topology.as_mdtraj())

view = nglview.show_mdtraj(traj)
view.add_ball_and_stick('all', radius=20.)
view

NGLWidget()

In [6]:
# Our implemented Integrator.
my_integrator = LangevinISPIntegrator(system=system, dt=0.05, gamma=5., T=300)

# Initialize the velocity.
my_integrator.initialize_velocities(temperature=300.)

# Propagate MD for 500 steps and print the temperature every 100 steps.
for i in range(500):
  my_integrator.step()

  if (i+1) % 100 == 0:
    print(f"Current step: {i+1}, temperature {my_integrator.compute_temperature():8.4f} K.")

# Propagate MD for another 500 steps and record the trajectory every 10 steps.
trajectory = list()

for i in range(500):
  my_integrator.step()

  if (i+1) % 10 == 0:
    trajectory.append( box.wrap(my_integrator.system.coordinates) )

# Visualize the trajectory using `MDTraj` and `NGLView`.

traj = mdtraj.Trajectory(xyz=np.asarray(trajectory), topology=system.topology.as_mdtraj())
view = nglview.show_mdtraj(traj)
view.add_ball_and_stick('all', radius=20.)
view

Current step: 100, temperature 365.7077 K.
Current step: 200, temperature 299.0289 K.
Current step: 300, temperature 287.9446 K.
Current step: 400, temperature 297.9352 K.
Current step: 500, temperature 328.2173 K.


NGLWidget(max_frame=49)


---

### 4. **Your Answer**

##### 4.1 **Derive the integration algorithm**

##### 4.2. **Implement the integrator**

In [ ]:
import numpy as np

import mdpy.utils
import mdpy.system
import mdpy.integrators

class YourCustomIntegrator(mdpy.integrators.Integrator):

  def __init__(self, 
               system: mdpy.system.System, 
               dt:     float, 
               T:      float, ):
    r"""The constructor for the Langevin ISP integrator.
    
      Args:
        system (mdpy.system.System): The `MDPy` system bounded to this integrator.
        dt     (float): The time step size (in ps).
        T      (float): The temperature (in K).
    """
    super().__init__(system)
    # Control paramters.
    self.dt = dt
    self.T = T
    
    # --------------------------------------------------------
    # TASK: you may pass in additional control arguments here.
    # --------------------------------------------------------




  
  def step(self) -> None:
    r"""Advance the integration by ONE time step."""
    # extract states from the system.
    m = self.system.topology.masses  # [N, 1], the mass        m
    x = self.system.coordinates      # [N, 3], the coordinates x(t)
    v = self.system.velocities       # [N, 3], the velocities  v(t-0.5dt)
    f = self.system.compute_forces() # [N, 3], the potential forces F
    
    # -----------------------------------------------
    # TASK: Implement the integration algorithm here. 
    # -----------------------------------------------




    # update states to the system.
    self.system.velocities  = v_next # Update velocities  v(t+0.5dt)
    self.system.coordinates = x_next # Update coordinates x(t+dt).

  def initialize_velocities(self, temperature: float) -> None:
    r"""Initialize the particle velocities per Maxwellian at the designated temperature.
    
      Args:
        temperature (float): The temperature (in K).
    """
    # -------------------------------------------------
    # TASK: Implement the velocity initialization here.
    #       Remember to shift the velocities to v(t) if
    #       your integrator uses the leap-frog scheme.
    # -------------------------------------------------




  def compute_kinetic_energy(self) -> float:
    r"""Compute the kinetic energy (in kcal/mol) of the system using the shifted velocities v(t).
    
      Returns:
        kinetic_energy (float): The kinetic energy (in kcal/mol) at v(t).
    """
    # ----------------------------------------------------
    # TASK: Implement the kinetic energy calculation here.
    #       Remember to shift the velocities to v(t) if 
    #       your integrator uses the leap-frog scheme.
    # ----------------------------------------------------



    return kinetic_energy

##### 4.3. **Test the integrator**

In [ ]:
# --------------------------------------------
# TASK: follow the examples provided above, 
#       test your implementation and visualize 
#       the dynamical trajectory.
# --------------------------------------------


